In [1]:
import os, re, sys, pickle

import pandas as pd
import numpy as np
from seestar import surveyInfoPickler
from seestar import AngleDisks

# Load in file info

In [11]:
fileinfo_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_fileinfo.pickle'
fileinfo = surveyInfoPickler.surveyInformation(fileinfo_path)

In [31]:
pointings = pd.read_csv(fileinfo.field_path)

# Check which fields already exist

In [22]:
files = os.listdir('/media/andy/UUI/Research/SF/SFdata/2MASS/Apogee-2MASS/')

item = "APOGEEfield_([0-9\.]*).csv"
files = [f for f in files if re.match(item, f)]

fields = [re.search(item, f).groups()[0] for f in files]
fields = np.array(fields).astype(float)
fields = fields.astype(int)

# Limit pointings to non-existent fields

In [5]:
done = pointings.fieldID.apply(lambda x: x in fields)
pointings_sub = pointings[~done]

# Save

In [64]:
pointings_sub.to_csv(fileinfo.field_path, index=False)

In [8]:
len(fields)

639

In [23]:
fields2 = np.array(pointings.fieldID)

In [24]:
f1 = [f for f in fields if f in fields2]
f2 = [f for f in fields2 if f in fields]
f3 = [f for f in fields2 if not f in fields]

In [25]:
len(f1), len(f2), len(f3)

(639, 639, 141)

# Move new files to UUI

In [14]:
files = ['/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv' for field in f3]

In [24]:
from shutil import copyfile
import sys

In [25]:
for f in files:
    fname = f.split('/')[-1]
    sys.stdout.write('\r'+fname)
    copyfile(f, '/media/andy/UUI/Research/SF/SFdata/2MASS/Apogee-2MASS/'+fname)

5203.csv

# Remove old files from photometric

In [31]:
files = ['/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv' for field in f1]

In [33]:
len(files)

639

In [34]:
for f in files:
    fname = f.split('/')[-1]
    sys.stdout.write('\r'+fname)
    os.remove(f)

5200.csv

# Copy old files in from UUI

In [35]:
#files = ['/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv' for field in f1]
files = ['/media/andy/UUI/Research/SF/SFdata/2MASS/Apogee-2MASS/APOGEEfield_'+str(field)+'.0.csv' for field in f1]

In [39]:
for field in f1:
    
    fold = '/media/andy/UUI/Research/SF/SFdata/2MASS/Apogee-2MASS/APOGEEfield_'+str(field)+'.0.csv'
    fnew = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv'
    
    copyfile(fold, fnew)

In [46]:
files = ['/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv' for field in f1]
dfold = pd.read_csv(files[0])

In [48]:
np.min(dfold.RA), np.max(dfold.RA), np.min(dfold.Dec), np.max(dfold.Dec)

(2.8331669169966633,
 2.8850186943046516,
 -0.030437739303325186,
 0.02159506355468096)

In [47]:
files = ['/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv' for field in f3]
dfnew = pd.read_csv(files[0])

In [49]:
np.min(dfnew.RA), np.max(dfnew.RA), np.min(dfnew.Dec), np.max(dfnew.Dec)

(5.288602783226168, 5.354602426343626, 0.6371634928479228, 0.6891547219631466)

In [47]:
files = ['/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'+str(field)+'.csv' for field in f1]

for f in files:
    sys.stdout.write('\r'+f)
    df = pd.read_csv(f)
    df['glon'], df['glat'] = AngleDisks.EquatToGal(df.RA, df.Dec)
    df.to_csv(f, index=False)

/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/2052.csv

# Null magnitude values

In [25]:
fpath = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/photometric/'
for f in os.listdir(fpath)[1:]:
    sys.stdout.write('\r'+f)
    df = pd.read_csv(fpath+f, low_memory=False)
    
    nulls = (df.k.astype(str)=='null')|(df.j.astype(str)=='null')|(df.h.astype(str)=='null')
    df = df[~nulls]
    df.j = df.j.astype(float)
    df.k = df.k.astype(float)
    df.h = df.h.astype(float)
    
    df.to_csv(fpath+f, index=False)

5200.csv

# Test subsample pointings

In [12]:
pointings_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_fieldinfofull.csv'
pointings = pd.read_csv(pointings_path)

In [3]:
#fieldsample = [4335, 4337, 4338, 4394, 4329, 5170]
pointings = pointings[390:]#pointings.fieldID.apply(lambda x: x in fieldsample)]

In [11]:
len(pointings)

390

In [13]:
pointings.to_csv(fileinfo.field_path, index=False)

In [24]:
df[df.fieldID==4288        # Number of Gaussian components
        if datatype == "spectro": nComponents = 1
        elif datatype == "photo": nComponents = 2

        # Generate the model
        model = StatisticalModels.GaussianEM(x=x, y=y, nComponents=nComponents, rngx=mag_range, rngy=col_range)
        #model = StatisticalModels.GaussianMM(x, y, nComponents, mag_range, col_range)
        # Add in SFxDF<DF constraint for the spectrograph distribution
        if datatype == "spectro": 
            model.photoDF, model.priorDF = (photoDF, True)
        model.runningL = False
        model.optimizeParams()
        # Test integral if you want to see the value/error in the integral when calculated
        # model.testIntegral()]

,APOGEE_ID,fieldID,glon,glat,s,Japp,Kapp,Happ,apogee_m_h,dm,...,par,log10age,mass,Z,R,phi,z,flag,ageGyr,mh
16895,2M03020992+3925149,4288,2.596427,-0.293304,4.132176,12.443,11.951,11.836,-1.117762,13.100174,...,0.240625,0.820497,0.988557,-1.001056,11.764991,-0.175058,-1.174905,False,6.614506,-1.001056
16909,2M03022304+3958392,4288,2.592004,-0.284530,3.095380,12.258,11.709,11.577,-0.181666,12.477036,...,0.320294,0.805918,1.131334,-0.142225,10.846813,-0.143425,-0.850566,False,6.396136,-0.142225
16919,2M03023102+3904445,4288,2.600637,-0.297862,3.388551,11.621,11.121,11.016,-0.419273,12.645914,...,0.295866,0.783907,1.107482,-0.369777,11.104591,-0.150758,-0.975721,False,6.080046,-0.369777
16992,2M03031831+3936241,4288,2.598182,-0.288569,4.298884,12.130,11.533,11.428,-0.623700,13.151774,...,0.234590,0.879305,1.000022,-0.568666,11.921791,-0.179601,-1.203356,False,7.573644,-0.568666
16993,2M03031914+4002539,4288,2.594170,-0.281892,1.079011,11.459,10.974,10.898,-0.430803,10.193374,...,0.915887,0.949136,0.980723,-0.382252,9.101521,-0.059289,-0.284640,False,8.894799,-0.382252
17007,2M03032702+3916043,4288,2.601745,-0.293423,2.721469,12.116,11.623,11.508,-0.415491,12.186811,...,0.366095,0.722125,1.142716,-0.367661,10.521793,-0.127515,-0.769316,False,5.273813,-0.367661
17018,2M03033393+3848558,4288,2.606284,-0.300030,2.481516,12.640,12.133,12.055,-0.373598,11.984542,...,0.401948,0.733699,1.136102,-0.339270,10.311602,-0.117475,-0.715926,False,5.416257,-0.339270
17025,2M03034284+3920395,4288,2.601840,-0.291822,2.081052,11.347,10.733,10.561,0.466454,11.601369,...,0.478658,0.811286,1.223952,0.469989,9.963587,-0.102963,-0.581793,False,6.475695,0.469989
17026,2M03034545+3931194,4288,2.600331,-0.289073,2.006187,10.931,10.398,10.282,-0.283661,11.490221,...,0.504263,0.651003,1.223147,-0.254215,9.898870,-0.100207,-0.555077,False,4.477164,-0.254215
17028,2M03034826+3846167,4288,2.607426,-0.300281,1.916921,12.110,11.591,11.507,-0.002625,11.425144,...,0.519652,0.569536,1.354451,0.002769,9.821502,-0.095009,-0.550312,False,3.711384,0.002769


In [ ]:
# Number of Gaussian components
if datatype == "spectro": nComponents = 1
elif datatype == "photo": nComponents = 2

# Generate the model
model = StatisticalModels.GaussianEM(x=x, y=y, nComponents=nComponents, rngx=mag_range, rngy=col_range)
#model = StatisticalModels.GaussianMM(x, y, nComponents, mag_range, col_range)
# Add in SFxDF<DF constraint for the spectrograph distribution
if datatype == "spectro": 
    model.photoDF, model.priorDF = (photoDF, True)
model.runningL = False
model.optimizeParams()
# Test integral if you want to see the value/error in the integral when calculated
# model.testIntegral()

In [42]:
pointings = pd.read_csv("/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_fieldinfofull.csv")

In [43]:
pointings[pointings.fieldID==5170]

,fieldID,glon,glat,halfangle,Magmin,Magmax,Colmin,Colmax
722,5170,3.363766,-0.214386,0.026005,NoLimit,NoLimit,NoLimit,NoLimit


In [44]:
pointings[(pointings.glon<3.37)&(pointings.glon>3.36)]

,fieldID,glon,glat,halfangle,Magmin,Magmax,Colmin,Colmax
567,2084,3.363629,0.402310,0.026005,NoLimit,NoLimit,NoLimit,NoLimit
722,5170,3.363766,-0.214386,0.026005,NoLimit,NoLimit,NoLimit,NoLimit


In [30]:
pointings.to_csv("/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_fieldinfo.csv", index=False)

In [36]:
for i in range(10):
    sys.stdout.write('\r Hello')
print('...hi')

sys.stdout.write('\rhey ho there')

 Hello...hi
hey ho there

In [37]:
x = np.array((1,2))
np.std(x)

0.5

In [46]:
import pickle
obsSF_pickle_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_obsSF.pickle'
with open(obsSF_pickle_path, "rb") as f:
    obsSF_dicts = pickle.load(f)

In [47]:
from seestar import SFInstanceClasses
for field in obsSF_dicts: # Load classes from dictionaries
    
    # Initialise class instance
    obsSF_field = SFInstanceClasses.observableSF(field)
    # Set class attributes from dictionary
    SFInstanceClasses.setattrs(obsSF_field, **obsSF_dicts[field])

    try: obsSF_field.DF_model['modelname']
    except TypeError: print(field)

4337
4338


In [48]:
# Initialise class instance
obsSF_field = SFInstanceClasses.observableSF(4337)
# Set class attributes from dictionary
SFInstanceClasses.setattrs(obsSF_field, **obsSF_dicts[field])

try: obsSF_field.DF_model['modelname']

{'distribution': <function seestar.StatisticalModels.bivGaussMix>,
 'modelname': 'GaussianEM',
 'mux': 8.43204273504273,
 'muy': 0.5601538461538463,
 'nComponents': 2,
 'param_shape': (2, 6),
 'params_f': array([[ 9.15644310e-01,  1.96939617e-01, -1.16156230e-01,
          8.38058076e-01,  1.17522019e+02,  1.27701300e-01],
        [-1.63816228e-01,  2.10140527e+00,  3.42482093e-01,
          1.00561116e+00,  6.87666743e+01, -1.72941287e-01]]),
 'params_i': array([[ 0.12145651,  1.        ,  0.50006514,  1.        , 58.        ,
          0.        ],
        [-0.70393515,  1.        , -0.51779721,  1.        , 58.        ,
          0.        ]]),
 'params_l': array([[-15.51993613,   0.        ,  -3.8956465 ,   0.        ,
           0.        ,         -inf],
        [-15.51993613,   0.        ,  -3.8956465 ,   0.        ,
           0.        ,         -inf]]),
 'params_u': array([[0.96591461,        inf, 3.51287593,        inf,        inf,
                inf],
        [0.96591461, 

In [ ]:
for field in obsSF_dicts: # Load classes from dictionaries
    
    # Initialise class instance
    obsSF_field = SFInstanceClasses.observableSF(field)
    # Set class attributes from dictionary
    SFInstanceClasses.setattrs(obsSF_field, **obsSF_dicts[field])
    print(field)
    obsSF_field.DF_model['modelname']

In [48]:
df = pd.read_csv(fileinfo.spectro_path)

In [22]:
import pickle
obsSF_pickle_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_obsSF.pickle'
with open(obsSF_pickle_path, "rb") as f:
    obsSF_dicts = pickle.load(f)

In [20]:
obsSF_pickle_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_obsSF1.pickle'
with open(obsSF_pickle_path, "rb") as f:
    obsSF_dicts1 = pickle.load(f)

In [24]:
obsSF_dicts1.update(obsSF_dicts)

In [23]:
i=0
for field in obsSF_dicts:
    i+=1
    
i

4

In [16]:
obsSF_pickle_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_obsSF.pickle'
with open(obsSF_pickle_path, 'wb') as handle:
    pickle.dump(obsSF_dicts, handle)
print("...done\n.")

...done
.


In [53]:
from seestar import SelectionGrid
from seestar import SFInstanceClasses

In [54]:
# Initialise class instance
SF = SelectionGrid.SFGenerator('/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_fileinfo.pickle')

Would you like the selection function in: a) observable, b) intrinsic, c) both? (return a, b or c)a
Path to observable SF (Apogee_new_obsSF.pickle) exists. Use this to ? (y/n)y
No field overlap information. This will be generated using the pointings data.

{'glon': 'phi', 'Colmin': 'Colmin', 'glat': 'theta', 'Colmax': 'Colmax', 'Magmin': 'Magmin', 'Magmax': 'Magmax', 'fieldID': 'fieldID', 'halfangle': 'halfangle'}
Unpickling colour-magnitude interpolant dictionaries...
...done.



In [66]:
coords = ['c', 'h']
x = np.array((0, 1, 2, 3))
y = np.array((0, 1, 2, 3))
df = pd.DataFrame(np.vstack((x, y)).T, columns=coords)

for field in SF.obsSF: # Load classes from dictionaries

    
    SFcalc = lambda field, df: np.array( SF.obsSF[field]((df[coords[0]], df[coords[1]])) )
    
    try: SFcalc(field, df)
    except TypeError: print(field)

4335
4337
4338


In [68]:
SF.obsSF[4335]((x, y))

TypeError: 'NoneType' object has no attribute '__getitem__'

In [7]:
float(20)/8

2.5

In [7]:
obsSF_pickle_path = '/media/andy/37E3-0F91/Research/SF/SFdata/Apogee_new/Apogee_new_obsSF2.pickle'
with open(obsSF_pickle_path, "rb") as f:
    obsSF_dicts = pickle.load(f)

In [8]:
i=0
for key in obsSF_dicts:
    i+=1
i

780

In [ ]:
for 